<a href="https://colab.research.google.com/github/AndersonDeFaro/Estudos-Alura-Pyhton/blob/master/99-Generative_AI/Coletadar_Dados_Leiloes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4
#'https://www.abaleiloes.com.br/leiloes'

## Teste uso de Web Scrap

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.abaleiloes.com.br/agenda'  # Substitua pela URL correta
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

data = []
for item in soup.select('.cont-leiloes .flex-leiloes li.col-md-3.col-sm-6.col-xs-12.isotope-item.proximos'):
    title = item.find('h3').text.strip() if item.find('h3') else None
    description = item.find('p').text.strip() if item.find('p') else None

    # Extract span tags from 'post-content' div
    post_content = item.find('div', class_='post-content')
    span_data = {}
    key_counts = {}  # Keep track of key occurrences
    if post_content:
        for span in post_content.find_all('span'):
            span_text = span.get_text(strip=True)
            # Assuming span text is in the format "Field Name: Value"
            field_name, value = span_text.split(':', 1) if ':' in span_text else (span_text, '')
            field_name = field_name.strip()
            value = value.strip()

            # Handle duplicate keys
            if field_name in key_counts:
                key_counts[field_name] += 1
                field_name = f"{field_name}_{key_counts[field_name]}"
            else:
                key_counts[field_name] = 0

            span_data[field_name] = value

    data.append({
        'title': title,
        'description': description,
        'span_details': span_data
    })

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
display(df)

,title,description,span_details
0,JUSTIÇA ESTADUAL DE GARARU/SE,Local: www.abaleiloes.com.br,"{'Leilão': '3', 'ABERTURA': '24/02/2023 às 10:..."
1,JUSTIÇA ESTADUAL DE ITABAIANINHA/SE,Local: www.abaleiloes.com.br,"{'Leilão': '4', 'ABERTURA': '27/02/2023 às 09:..."
2,JUSTIÇA ESTADUAL DE TOBIAS BARRETO/SE,Local: www.abaleiloes.com.br,"{'Leilão': '5', 'ABERTURA': '27/02/2023 às 10:..."
3,JUSTIÇA ESTADUAL DA COMARCA DE ESTÂNCIA/SE,Local: www.abaleiloes.com.br,"{'Leilão': '6', 'ABERTURA': '28/02/2023 às 10:..."
4,JUSTIÇA ESTADUAL DA LAGARTO/SE,Local: www.abaleiloes.com.br,"{'Leilão': '7', 'ABERTURA': '28/02/2023 às 11:..."
...,...,...,...
181,JUSTIÇA ESTADUAL DE CARIRA/SE,Local: www.abaleiloes.com.br,"{'Leilão': '196', 'ABERTURA': '31/07/2024 às 1..."
182,JUSTIÇA ESTADUAL DE PORTO DA FOLHA/SE,Local: www.abaleiloes.com.br,"{'Leilão': '190', 'ABERTURA': '13/08/2024 às 0..."
183,JUSTIÇA ESTADUAL DE INDIAROBA/SE,Local: www.abaleiloes.com.br,"{'Leilão': '78', 'ABERTURA': '14/08/2024 às 09..."
184,JUSTIÇA ESTADUAL DE ESTÂNCIA/SE,Local: www.abaleiloes.com.br,"{'Leilão': '202', 'ABERTURA': '14/08/2024 às 1..."


## Escrever resultado em Google Sheet

In [30]:
# --- Configuração do Google Drive ---
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
from googleapiclient.discovery import build
import time

creds, _ = default()
drive_service = build('drive', 'v3', credentials=creds)
gc = gspread.authorize(creds)

def find_folder_id(drive_service, folder_path):
    """Encontra o ID da pasta especificada."""
    folders = folder_path.split('/')
    parent_id = 'root'
    for folder in folders:
        query = f"name='{folder}' and mimeType='application/vnd.google-apps.folder' and '{parent_id}' in parents"
        results = drive_service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
        items = results.get('files', [])
        if items:
            parent_id = items[0]['id']
        else:
            return None
    return parent_id

folder_path = "Colab Notebooks/Dados"
folder_id = find_folder_id(drive_service, folder_path)
file_name = 'aba_leiloes.xlsx'

# --- Criação/recriação da planilha ---
if folder_id:
    print(f"ID da pasta '{folder_path}': {folder_id}")

    # Verifica se o arquivo existe e o exclui se existir
    query = f"name='{file_name}' and '{folder_id}' in parents"
    results = drive_service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])
    if items:
        file_id = items[0]['id']
        drive_service.files().delete(fileId=file_id).execute()
        print(f"Arquivo '{file_name}' excluído.")

    # Cria um novo arquivo na pasta especificada
    file_metadata = {
        'name': file_name,
        'parents': [folder_id],
        'mimeType': 'application/vnd.google-apps.spreadsheet'
    }
    spreadsheet = gc.create(file_metadata['name'], folder_id)
    print(f"Arquivo '{file_name}' criado.")

    # --- Escrevendo dados na planilha ---
    worksheet = spreadsheet.sheet1

    # Escreve o cabeçalho
    header = df.columns.tolist()
    worksheet.append_row(header)

    # Escreve os dados com pausa para evitar atingir o limite de taxa
    count = 0
    for _, row in df.iterrows():
        row_values = row.tolist()
        # Convert all values to strings to avoid data type issues
        row_values = [str(value) for value in row_values]
        worksheet.append_row(row_values)
        count += 1
        if count % 100 == 0:  # Pausa de 1 segundo a cada 100 escritas
            time.sleep(1)

    print("Dados gravados na planilha com sucesso!")

else:
    print(f"Pasta '{folder_path}' não encontrada.")

ID da pasta 'Colab Notebooks/Dados': 12r9bwgrvusRtYHudnYaZJG5U_b0Qmhro
Arquivo 'aba_leiloes.xlsx' excluído.
Arquivo 'aba_leiloes.xlsx' criado.
Dados gravados na planilha com sucesso!
